# 9-Step Rhetorical Evaluation Notebook

This notebook runs the "From Evaluation to Growth" analysis framework interactively.

## The 9 Steps

### Phase 1: EVALUATION
1. **Critique** - Strengths and weaknesses assessment
2. **Logic Check** - Internal consistency and argument flow
3. **Logos Review** - Rational appeal (evidence, facts)
4. **Pathos Review** - Emotional resonance
5. **Ethos Review** - Credibility and authority markers

### Phase 2: RISK
6. **Blind Spots** - Overlooked areas and assumptions
7. **Shatter Points** - Vulnerabilities and weak arguments

### Phase 3: GROWTH
8. **Bloom** - Emergent insights and connections
9. **Evolve** - Synthesized improvement recommendations

## Setup

In [ ]:
import sys
from pathlib import Path
import json

# Add framework to path
FRAMEWORK_ROOT = Path.cwd().parent.parent.parent
sys.path.insert(0, str(FRAMEWORK_ROOT))

# Import framework modules
from framework.core import Atomizer, Corpus, AtomLevel
from framework.analysis import EvaluationAnalysis

print(f"Framework root: {FRAMEWORK_ROOT}")
print("Evaluation module loaded successfully!")

In [ ]:
# Configuration
PROJECT_DIR = Path.cwd().parent
PROJECT_NAME = PROJECT_DIR.name

RAW_DIR = PROJECT_DIR / "data" / "raw"
PROCESSED_DIR = PROJECT_DIR / "data" / "processed"
CORPUS_FILE = RAW_DIR / f"{PROJECT_NAME}_atomized.json"

print(f"Project: {PROJECT_NAME}")
print(f"Corpus: {CORPUS_FILE}")

## Load Corpus

In [ ]:
# Load corpus
if CORPUS_FILE.exists():
    corpus = Atomizer.load_json(CORPUS_FILE)
    print(f"✅ Loaded corpus: {corpus.name}")
    print(f"   Themes: {corpus.count_atoms(AtomLevel.THEME)}")
    print(f"   Sentences: {corpus.count_atoms(AtomLevel.SENTENCE)}")
else:
    print("❌ Corpus not found. Run atomization first.")

## Initialize Evaluation Module

In [ ]:
# Create evaluation module instance
evaluator = EvaluationAnalysis()
print(f"✅ {evaluator.name}: {evaluator.description}")

## Optional: Configure LLM Provider

For deeper insights on Critique, Bloom, and Evolve steps, you can configure an LLM provider.

In [ ]:
# LLM Configuration (optional - set to None to skip)
LLM_CONFIG = None  # Set to enable LLM-powered insights

# Example configurations:
# 
# Anthropic:
# LLM_CONFIG = {
#     "provider": "anthropic",
#     "model": "claude-sonnet-4-20250514",
#     "api_key_env": "ANTHROPIC_API_KEY"
# }
# 
# OpenAI:
# LLM_CONFIG = {
#     "provider": "openai",
#     "model": "gpt-4o",
#     "api_key_env": "OPENAI_API_KEY"
# }
# 
# Ollama (local):
# LLM_CONFIG = {
#     "provider": "ollama",
#     "model": "llama3"
# }

if LLM_CONFIG:
    print(f"LLM configured: {LLM_CONFIG['provider']} / {LLM_CONFIG['model']}")
else:
    print("LLM not configured - running heuristic analysis only")

## Run Full Analysis

In [ ]:
# Configure which steps to run (1-9, or subset)
STEPS_TO_RUN = [1, 2, 3, 4, 5, 6, 7, 8, 9]  # All steps

# Run analysis
config = {
    "steps": STEPS_TO_RUN,
}
if LLM_CONFIG:
    config["llm"] = LLM_CONFIG

print("Running evaluation analysis...")
print(f"Steps: {STEPS_TO_RUN}\n")

result = evaluator.analyze(corpus, config=config)

print("\n✅ Analysis complete!")

## View Overall Results

In [ ]:
# Display summary
summary = result.data.get("summary", {})
overall_score = summary.get("overall_score", 0)
phase_scores = summary.get("phase_scores", {})

print("\n" + "=" * 60)
print("📊 EVALUATION SUMMARY")
print("=" * 60)

# Score bar visualization
def score_bar(score, width=30):
    filled = int(score / 100 * width)
    return "█" * filled + "░" * (width - filled)

print(f"\nOverall Score: {overall_score:.1f}/100")
print(f"[{score_bar(overall_score)}]")

print("\nPhase Scores:")
for phase, score in phase_scores.items():
    emoji = {"evaluation": "🔍", "risk": "⚠️", "growth": "🌱"}.get(phase, "📌")
    print(f"  {emoji} {phase.title():12} {score:5.1f} [{score_bar(score, 20)}]")

## View Step Details

In [ ]:
# Display each step result
phases = result.data.get("phases", {})

STEP_ICONS = {
    "critique": "🔍",
    "logic_check": "🧠",
    "logos": "📊",
    "pathos": "💓",
    "ethos": "👤",
    "blind_spots": "👁️",
    "shatter_points": "💥",
    "bloom": "🌸",
    "evolve": "🚀",
}

for phase_name, phase_data in phases.items():
    print(f"\n{'='*60}")
    print(f"Phase: {phase_name.upper()}")
    print(f"{'='*60}")
    
    for step_name, step_data in phase_data.items():
        icon = STEP_ICONS.get(step_name, "📌")
        score = step_data.get("score", 0)
        step_num = step_data.get("step_number", 0)
        
        print(f"\n{icon} Step {step_num}: {step_name.replace('_', ' ').title()}")
        print(f"   Score: {score:.1f}/100 [{score_bar(score, 15)}]")
        
        # Show key metrics
        metrics = step_data.get("metrics", {})
        if metrics:
            print("   Metrics:")
            for k, v in list(metrics.items())[:4]:
                print(f"      • {k.replace('_', ' ').title()}: {v}")

## View Findings

In [ ]:
# Collect all findings
all_findings = []

for phase_name, phase_data in phases.items():
    for step_name, step_data in phase_data.items():
        for finding in step_data.get("findings", []):
            finding["step"] = step_name
            all_findings.append(finding)

# Group by type
TYPE_EMOJI = {
    "strength": "✅",
    "weakness": "⚠️",
    "blind_spot": "👁️",
    "shatter_point": "💥",
    "insight": "💡",
    "observation": "📝",
}

print("\n" + "=" * 60)
print("🔎 KEY FINDINGS")
print("=" * 60)

# Show strengths
strengths = [f for f in all_findings if f.get("type") == "strength"]
if strengths:
    print("\n✅ Strengths:")
    for f in strengths:
        print(f"   • [{f['step']}] {f.get('description', '')}")

# Show weaknesses
weaknesses = [f for f in all_findings if f.get("type") == "weakness"]
if weaknesses:
    print("\n⚠️ Weaknesses:")
    for f in weaknesses:
        print(f"   • [{f['step']}] {f.get('description', '')}")

# Show insights
insights = [f for f in all_findings if f.get("type") == "insight"]
if insights:
    print("\n💡 Insights:")
    for f in insights:
        print(f"   • [{f['step']}] {f.get('description', '')}")

## View Recommendations

In [ ]:
# Top recommendations
top_recs = summary.get("top_recommendations", [])

print("\n" + "=" * 60)
print("📋 TOP RECOMMENDATIONS")
print("=" * 60)

for i, rec in enumerate(top_recs[:10], 1):
    print(f"\n{i}. {rec}")

## Export Results

In [ ]:
# Save results to processed directory
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
output_file = PROCESSED_DIR / "evaluation_data.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result.to_dict(), f, indent=2, ensure_ascii=False)

print(f"\n✅ Results saved to: {output_file}")

## Run Individual Steps

You can also run individual steps for deeper analysis.

In [ ]:
# Run only specific steps
# Modify STEPS_TO_RUN to run a subset

# Example: Run only Logos, Pathos, Ethos (the rhetorical appeals)
APPEALS_ONLY = [3, 4, 5]

appeals_result = evaluator.analyze(corpus, config={"steps": APPEALS_ONLY})

print("\n📊 Rhetorical Appeals Analysis")
print("=" * 40)

appeals_phases = appeals_result.data.get("phases", {})
eval_phase = appeals_phases.get("evaluation", {})

for step_name in ["logos", "pathos", "ethos"]:
    if step_name in eval_phase:
        score = eval_phase[step_name].get("score", 0)
        icon = STEP_ICONS.get(step_name, "📌")
        print(f"{icon} {step_name.title():8} {score:5.1f} [{score_bar(score, 20)}]")

---

## Next Steps

1. **Visualize results**: Open `visualization.ipynb` or run:
   ```bash
   lingframe visualize -p <project-name>
   ```

2. **Export report**: The JSON results can be used to generate custom reports

3. **Iterate**: Make improvements based on recommendations and re-run analysis